# MongoDB Exercise

In [1]:
from pymongo import MongoClient
import json
import urllib.parse

with open('credentials_mongodb.json') as f:
    login = json.load(f)

username = login['username']
password = urllib.parse.quote(login['password'])
host = login['host']
url = "mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority".format(username, password, host)

In [2]:
client = MongoClient(url)

List all db names collections

In [3]:
client.list_database_names()

['sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

Just checking data; I can also check schema in Mongo compass.

In [4]:
client.sample_mflix.movies.find_one()

{'_id': ObjectId('573a1390f29313caabcd42e8'),
 'plot': 'A group of bandits stage a brazen train hold-up, only to find a determined posse hot on their heels.',
 'genres': ['Short', 'Western'],
 'runtime': 11,
 'cast': ['A.C. Abadie',
  "Gilbert M. 'Broncho Billy' Anderson",
  'George Barnes',
  'Justus D. Barnes'],
 'poster': 'https://m.media-amazon.com/images/M/MV5BMTU3NjE5NzYtYTYyNS00MDVmLWIwYjgtMmYwYWIxZDYyNzU2XkEyXkFqcGdeQXVyNzQzNzQxNzI@._V1_SY1000_SX677_AL_.jpg',
 'title': 'The Great Train Robbery',
 'fullplot': "Among the earliest existing films in American cinema - notable as the first film that presented a narrative story to tell - it depicts a group of cowboy outlaws who hold up a train and rob the passengers. They are then pursued by a Sheriff's posse. Several scenes have color included - all hand tinted.",
 'languages': ['English'],
 'released': datetime.datetime(1903, 12, 1, 0, 0),
 'directors': ['Edwin S. Porter'],
 'rated': 'TV-G',
 'awards': {'wins': 1, 'nominations': 0, 

For the following questions you want to use database `sample_mflix` and collection `movies`.

***Question 1:*** 

Write a query to count the number of movies casted either  'Tommy Lee Jones' or 'Will Smith' with `num_mflix_comments` greater than 120.

Hint: Make sure you select 'type' as 'movie' - this is applicable for all questions where we are asking about movies.

In [5]:
movies_collection = client['sample_mflix']['movies']
print(movies_collection.count_documents({
        "cast": {"$in": ["Tommy Lee Jones", "Will Smith"]},
        "num_mflix_comments": {"$gt": 120},
        "type": "movie",
}))

8


***Question 2:*** 

Write a query to count the number of movies casted either 'Tommy Lee Jones' or 'Will Smith' with `num_mflix_comments` greater than 120 or `runtime` less than 100 min.

Hint: You can develop on the Question 1 query to include the `or` condition for `num_mflix_comments` or `runtime`.

In [6]:
movies_collection = client['sample_mflix']['movies']
print(movies_collection.count_documents({
        "cast": {"$in": ["Tommy Lee Jones", "Will Smith"]},
        "$or": [
            {"num_mflix_comments": {"$gt": 120}},
            {"runtime": {"$lt": 100}}
        ],
        "type": "movie",
}))

12


***Question 3:*** 

Write a query to select distinct movie titles casted either 'Tommy Lee Jones' or 'Will Smith' with `nominations` greater than or equal to 10.

Hint: You can develop the Question 1 query. Takeout condition on `num_mflix_comments` and add `nominations` instead. But how can you add `nominations` ? Can you access `nominations` by just giving `nominations` ? What is special about the `nominations` field? You can check mongo compass schema tab or just query to see data `client.sample_mflix.movies.find_one()`

Remember, unlike the previous 2 questions, here we are not asking for a count but to print the data. 

In [7]:
res = list(
    movies_collection.distinct("title", {
        "cast": {"$in": ["Tommy Lee Jones", "Will Smith"]},
        "awards.nominations": {"$gte": 10},
        "type": "movie",
    })
)
for r in res:
    print(r)

A Prairie Home Companion
Ali
Bad Boys II
Batman Forever
Captain America: The First Avenger
Coal Miner's Daughter
Enemy of the State
Hancock
Hitch
I Am Legend
I, Robot
In the Valley of Elah
Independence Day
Men in Black
Men in Black 3
No Country for Old Men
Shark Tale
The Fugitive
The Homesman
The Legend of Bagger Vance
The Pursuit of Happyness
Wild Wild West


***Question 4:*** 

Write a query to count the number of movies casted 'Norman Kerry' with `tomatoes.viewer.meter` greater than or equal to 5 and the field `languages` exists. 

Hint: 
- Make sure you don't include documents with a blank space in their `tomatoes.viewer.meter` field! ('$ne': '')
- You have to make sure that you use a keyword (check lecture notes to see what keyword is that) to make sure that the field `languages` exist. 

In [8]:
movies_collection = client['sample_mflix']['movies']
print(movies_collection.count_documents({
    "cast": "Norman Kerry",
    "tomatoes.viewer.meter": {"$gte": 5, "$ne": ""},
    "languages": {"$exists": True},
    "type": "movie",
}))

0


In [9]:
movies_collection = client['sample_mflix']['movies']

cursor = movies_collection.distinct("title", {
    "languages": {"$eq": ["English", "Spanish", "French", "German"]}, "type": "movie",
})

print(list(cursor))

['Hètel Terminus', 'Sorcerer', 'Viva Maria!']
